<a href="https://colab.research.google.com/github/Sameer-30/Dynamic-Response-Comparison-of-Damped-Systems/blob/main/Comparison_between_analytical_and_numerical_solutions_of_a_damped_Single_Degree_of_Freedom_(SDOF)_system_subjected_to_a_simple_half_sine_seismic_pulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad

# 1) SYSTEM & PULSE PARAMETERS
m = 1.0
omega_n = 2 * np.pi
k = m * omega_n**2

A = 1.0      # half‑sine amplitude (m/s^2)
Tp = 0.5     # pulse duration (s)

t_final = 2.0
dt = 0.001
t = np.arange(0, t_final + dt, dt)
N = len(t)

beta, gamma = 1/4, 1/2  # Newmark parameters

# 2) GROUND ACCELERATION FUNCTION
def ground_acc(ti):
    return A * np.sin(np.pi * ti / Tp) * (ti <= Tp)

# 3) ANALYTICAL RESPONSE (Duhamel)
def analytical_response(zeta):
    omega_d = omega_n * np.sqrt(1 - zeta**2)
    u = np.zeros(N)
    for i, ti in enumerate(t):
        tau_max = min(ti, Tp)
        if tau_max > 0:
            integral, _ = quad(
                lambda tau: (1/omega_d)
                            * np.exp(-zeta*omega_n*(ti - tau))
                            * np.sin(omega_d*(ti - tau))
                            * ground_acc(tau),
                0, tau_max
            )
            u[i] = -integral
    return u

# 4) NUMERICAL RESPONSE (Newmark-β)
def newmark_response(zeta):
    c = 2*zeta*np.sqrt(k*m)

    # Precompute Newmark constants
    a0 = 1/(beta*dt**2)
    a1 = gamma/(beta*dt)
    a2 = 1/(beta*dt)
    a3 = 1/(2*beta) - 1
    a4 = gamma/beta - 1
    a5 = dt*(gamma/(2*beta) - 1)
    k_eff = k + a1*c + a0*m

    u = np.zeros(N)
    v = np.zeros(N)
    a = np.zeros(N)
    # initial acceleration
    a[0] = (-c*v[0] - k*u[0] - m*ground_acc(0)) / m

    for i in range(1, N):
        p_eff = (
            -m*ground_acc(t[i])
            + m*(a0*u[i-1] + a2*v[i-1] + a3*a[i-1])
            + c*(a1*u[i-1] + a4*v[i-1] + a5*a[i-1])
        )
        u[i] = p_eff / k_eff
        a[i] = a0*(u[i] - u[i-1]) - a2*v[i-1] - a3*a[i-1]
        v[i] = v[i-1] + dt*((1-gamma)*a[i-1] + gamma*a[i])

    return u

# 5) RUN & PLOT FOR VARIOUS DAMPING RATIOS
zeta_values = [0.02, 0.05, 0.10]
peak_analytical = []
peak_numerical = []

plt.figure(figsize=(12, 8))

# Time‐history subplot
plt.subplot(2,1,1)
for z in zeta_values:
    u_ana = analytical_response(z)
    u_num = newmark_response(z)
    peak_analytical.append(np.max(np.abs(u_ana)))
    peak_numerical.append(np.max(np.abs(u_num)))
    plt.plot(t, u_ana,   label=f'Analytical (ζ={z})')
    plt.plot(t, u_num, '--', label=f'Numerical (ζ={z})')

plt.title('Displacement Time‑Histories')
plt.xlabel('Time (s)')
plt.ylabel('Displacement (m)')
plt.legend()
plt.grid(True)

# Peak vs damping subplot
plt.subplot(2,1,2)
plt.plot(zeta_values, peak_analytical, 'o-', label='Analytical')
plt.plot(zeta_values, peak_numerical, 's--', label='Numerical')
plt.title('Peak Displacement vs. Damping Ratio')
plt.xlabel('Damping Ratio ζ')
plt.ylabel('Peak Displacement (m)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad

# (Re‑use your existing functions and parameters)
m = 1.0
omega_n = 2 * np.pi
k = m * omega_n**2
A, Tp = 1.0, 0.5
t_final, dt = 2.0, 0.001
t = np.arange(0, t_final + dt, dt)
beta, gamma = 1/4, 1/2

def ground_acc(ti):
    return A * np.sin(np.pi * ti / Tp) * (ti <= Tp)

def analytical_response(zeta):
    omega_d = omega_n * np.sqrt(1 - zeta**2)
    u = np.zeros_like(t)
    for i, ti in enumerate(t):
        tau_max = min(ti, Tp)
        if tau_max > 0:
            I, _ = quad(
                lambda tau: (1/omega_d)
                            * np.exp(-zeta*omega_n*(ti - tau))
                            * np.sin(omega_d*(ti - tau))
                            * ground_acc(tau),
                0, tau_max
            )
            u[i] = -I
    return u

def newmark_response(zeta):
    c = 2*zeta*np.sqrt(k*m)
    a0 = 1/(beta*dt**2); a1 = gamma/(beta*dt)
    a2 = 1/(beta*dt); a3 = 1/(2*beta)-1
    a4 = gamma/beta-1; a5 = dt*(gamma/(2*beta)-1)
    k_eff = k + a1*c + a0*m

    u = np.zeros_like(t)
    v = np.zeros_like(t)
    a = np.zeros_like(t)
    a[0] = (-c*v[0] - k*u[0] - m*ground_acc(0)) / m

    for i in range(1, len(t)):
        p_eff = (
            -m*ground_acc(t[i])
            + m*(a0*u[i-1] + a2*v[i-1] + a3*a[i-1])
            + c*(a1*u[i-1] + a4*v[i-1] + a5*a[i-1])
        )
        u[i] = p_eff / k_eff
        a[i] = a0*(u[i] - u[i-1]) - a2*v[i-1] - a3*a[i-1]
        v[i] = v[i-1] + dt*((1-gamma)*a[i-1] + gamma*a[i])
    return u

# Choose damping ratios
zeta_values = [0.02, 0.05, 0.10]

# Containers for error metrics
rmse = []
max_err = []

# Compute errors
for z in zeta_values:
    u_ana = analytical_response(z)
    u_num = newmark_response(z)
    err = u_num - u_ana
    rmse.append(np.sqrt(np.mean(err**2)))
    max_err.append(np.max(np.abs(err)))

# 1) Time‑history of error for ζ=0.05
z0 = 0.05
err0 = newmark_response(z0) - analytical_response(z0)
plt.figure()
plt.plot(t, err0)
plt.xlabel('Time (s)')
plt.ylabel('Error $u_{num}-u_{ana}$ (m)')
plt.title(f'Displacement Error Time History (ζ={z0})')
plt.grid(True)

# 2) RMSE and max‑error vs. ζ
plt.figure()
plt.plot(zeta_values, rmse, 'o-', label='RMSE')
plt.plot(zeta_values, max_err, 's--', label='Max abs error')
plt.xlabel('Damping Ratio ζ')
plt.ylabel('Error (m)')
plt.title('Error Metrics vs. Damping Ratio')
plt.legend()
plt.grid(True)

plt.show()
